In [ ]:
import torch
from diffusers import AutoPipelineForInpainting, LCMScheduler
from diffusers.utils import load_image, make_image_grid

pipe = AutoPipelineForInpainting.from_pretrained(
    "diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

# set scheduler
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# load LCM-LoRA
pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl")
pipe.fuse_lora()



In [ ]:
# load base and mask image
init_image = load_image("./image/image.png")
init_size = init_image.size
init_image = load_image("./image/image.png").resize((1024, 1024))
mask_image = load_image("./image/mask.png").resize((1024, 1024))

prompt = "sword"
neg_prompt = "food, sfw, plate"
image = pipe(
    prompt=prompt,
    negative_prompt=neg_prompt,
    image=init_image,
    mask_image=mask_image,
    num_inference_steps=5,
    guidance_scale=4,
).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)

In [ ]:
image = image.resize(init_size)
image

In [ ]:
init_image.size